In [2]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import skimage
import scipy
from skimage import transform
import PIL
from PIL import Image
import sklearn
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import transforms,datasets
import torchvision.models as models

In [3]:
#one way from tut and chatGPT

# NOTE: transformation defined for this classification task, data augmentation techniques used during training.
#       To understand each transformation, please read https://pytorch.org/vision/stable/transforms.html
#       - (Normalize data to range between [-1, 1])
#       - (Randomly flip the image left to right)
#       - (Zero-pad 4 pixels on each side and randomly crop 28x28 as input)
# train_transforms = T.Compose([
#     T.ToTensor(),
#     T.Normalize((0.5,), (0.5,)),
#     T.RandomCrop(size=(28, 28), padding=4),
# ])

# test_transforms = T.Compose([
#     T.ToTensor(),
#     T.Normalize((0.5,), (0.5,)),
# ])

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]),
}


In [4]:
class DomainNetDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, domain, split='train', transform=None):
        self.root_dir = root_dir
        self.domain = domain
        self.split = split
        self.transform = transform
        self.data = datasets.ImageFolder(root=root_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]
        return img, label

# Dataset paths
real_train_path = 'data/real_train'
real_test_path = 'data/real_test'
sketch_train_path = 'data/sketch_train'
sketch_test_path = 'data/sketch_test'

# Create datasets
real_train_dataset = DomainNetDataset(root_dir=real_train_path, domain='real', split='train', transform=data_transforms['train'])
real_test_dataset = DomainNetDataset(root_dir=real_test_path, domain='real', split='test', transform=data_transforms['test'])
sketch_train_dataset = DomainNetDataset(root_dir=sketch_train_path, domain='sketch', split='train', transform=data_transforms['train'])
sketch_test_dataset = DomainNetDataset(root_dir=sketch_test_path, domain='sketch', split='test', transform=data_transforms['test'])


In [ ]:
# Load pretrained ResNet-34 model
resnet34 = models.resnet34(pretrained=True)

# Modify final fully-connected layer
num_classes = 10  # Number of classes for this task
num_ftrs = resnet34.fc.in_features
resnet34.fc = nn.Linear(num_ftrs, num_classes)

/Applications/anaconda/anaconda3/envs/pythonforcomputervision/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Applications/anaconda/anaconda3/envs/pythonforcomputervision/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


: 

In [26]:
#one way from lecture
# 创建自定义数据集
training_data = CustomDataset(data_path='data/real_train/tree', kind='train')

# 创建数据加载器，并指定批次大小
batch_size = 32
data_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

# 循环迭代数据加载器，加载批次数据
for batch_idx, (images, labels) in enumerate(data_loader):
    # 在这里进行模型训练等操作
    pass

FileNotFoundError: [Errno 2] No such file or directory: 'data/real_train/tree/data_batch_1'

In [8]:
labels_map = {
    0: "tree",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck",
}
# helper function to load CIFAR-10 data
def load_data(data_path, file_name):
    """Load CIFAR-10 data from given data_path"""
    # file_path = os.path.join(data_path, file_name)
    file_path = data_path + '/' + file_name  # 使用字符串拼接构建文件路径
    with open(file_path, 'rb') as f:
        # load data with pickle
        entry = pickle.load(f, encoding='latin1')
        images = entry['data']
        labels = entry['labels']
        # reshape the array to 32x32 color image
        images = images.reshape(-1, 3, 32, 32)  # '-1' means the value will be deduced from the shape of array
    return images, labels

class CustomDataset(Dataset):
    def __init__(self, data_path, kind='train'):
        super().__init__()
        train_list = ['data_batch_1', 'data_batch_2','data_batch_3','data_batch_4']
        valid_list = ['data_batch_5']
        test_list = ['test_batch']

        if kind == 'train':
            self.file_list = train_list
        elif kind == 'valid':
            self.file_list = valid_list
        else:
            self.file_list = test_list
        self.images = []
        self.labels = []
        # for file_name in self.file_list:
        #     file_path = os.path.join(data_path, file_name)
        #     images, labels = load_data(data_path, file_name)
        #     self.images.append(images)
        #     self.labels.append(labels)

        for file_name in self.file_list:
            file_path = data_path + '/' + file_name  # 使用字符串拼接构建文件路径
            images, labels = load_data(data_path, file_name)
            self.images.append(images)
            self.labels.append(labels)

        # concatenate the data together
        self.images = np.concatenate(self.images, axis=0)   # NCHW
        self.images = self.images.transpose((0, 2, 3, 1))   # convert to HWC
        self.labels = np.concatenate(self.labels, axis=0)
   
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
       image = self.images[idx]
       label = self.labels[idx]
       
       return image, label 
    


### Visualize a data sample

In [11]:
training_data = CustomDataset(data_path='data/real_train/tree', kind='train')
figure = plt.figure(figsize=(8,8))
cols, rows = 3, 3
for i in range(1, cols*rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    ax = figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis('off')
    ax.imshow(img.squeeze())
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'data/real_train/tree/data_batch_1'